## Relevant imports and Groq Client

We start by importing all the libraries we'll be using in this tutorial as well as the Groq client.

In [1]:
import os
import re
import math
import json
from dotenv import load_dotenv

from groq import Groq
from openai import OpenAI

from tool_pattern.tool import tool
from utils.extraction import extract_tag_content

In [2]:
# Remember to load the environment variables. You should have the Groq API Key in there :)
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")
# google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if groq_api_key:
    print(f"groq API Key exists and begins {groq_api_key[:7]}")
else:
    print("Groq API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key not set
groq API Key exists and begins sk-ant-


In [3]:
LLM = "OPENAI"

In [4]:
if LLM == "OPENAI":
    client = OpenAI()
    MODEL = "gpt-4o-mini"
else:
    client = Groq()
    MODEL = "llama-3.1-70b-versatile"

In [5]:
@tool
def sum_two_elements(a: int, b: int) -> int:
    """
    Computes the sum of two integers.

    Args:
        a (int): The first integer to be summed.
        b (int): The second integer to be summed.

    Returns:
        int: The sum of `a` and `b`.
    """
    return a + b


@tool
def multiply_two_elements(a: int, b: int) -> int:
    """
    Multiplies two integers.

    Args:
        a (int): The first integer to multiply.
        b (int): The second integer to multiply.

    Returns:
        int: The product of `a` and `b`.
    """
    return a * b


@tool
def compute_log(x: int) -> float | str:
    """
    Computes the logarithm of an integer `x` with an optional base.

    Args:
        x (int): The integer value for which the logarithm is computed. Must be greater than 0.

    Returns:
        float: The logarithm of `x` to the specified `base`.
    """
    if x <= 0:
        return "Logarithm is undefined for values less than or equal to 0."

    return math.log(x)


available_tools = {
    "sum_two_elements": sum_two_elements,
    "multiply_two_elements": multiply_two_elements,
    "compute_log": compute_log,
}

In [6]:
# Define the System Prompt as a constant
REACT_SYSTEM_PROMPT = """
You are a function calling AI model. You operate by running a loop with the following steps: Thought, Action, Observation.
You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don' make assumptions about what values to plug
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.

For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>, "id": <monotonically-increasing-id>}
</tool_call>

Here are the available tools / actions:

<tools> 
%s
</tools>

Example session:

<question>What's the current temperature in Madrid?</question>
<thought>I need to get the current weather in Madrid</thought>
<tool_call>{"name": "get_current_weather","arguments": {"location": "Madrid", "unit": "celsius"}, "id": 0}</tool_call>

You will be called again with this:

<observation>{0: {"temperature": 25, "unit": "celsius"}}</observation>

You then output:

<response>The current temperature in Madrid is 25 degrees Celsius</response>

Additional constraints:

- If the user asks you something unrelated to any of the tools above, answer freely enclosing your answer with <response></response> tags.
"""

In [7]:
print("Tool name: ", sum_two_elements.name)
print("Tool signature: ", sum_two_elements.fn_signature)

Tool name:  sum_two_elements
Tool signature:  {"name": "sum_two_elements", "description": "\n    Computes the sum of two integers.\n\n    Args:\n        a (int): The first integer to be summed.\n        b (int): The second integer to be summed.\n\n    Returns:\n        int: The sum of `a` and `b`.\n    ", "parameters": {"properties": {"a": {"type": "int"}, "b": {"type": "int"}}}}


In [8]:
tools_signature = (
    sum_two_elements.fn_signature
    + ",\n"
    + multiply_two_elements.fn_signature
    + ",\n"
    + compute_log.fn_signature
)

In [9]:
print(tools_signature)

{"name": "sum_two_elements", "description": "\n    Computes the sum of two integers.\n\n    Args:\n        a (int): The first integer to be summed.\n        b (int): The second integer to be summed.\n\n    Returns:\n        int: The sum of `a` and `b`.\n    ", "parameters": {"properties": {"a": {"type": "int"}, "b": {"type": "int"}}}},
{"name": "multiply_two_elements", "description": "\n    Multiplies two integers.\n\n    Args:\n        a (int): The first integer to multiply.\n        b (int): The second integer to multiply.\n\n    Returns:\n        int: The product of `a` and `b`.\n    ", "parameters": {"properties": {"a": {"type": "int"}, "b": {"type": "int"}}}},
{"name": "compute_log", "description": "\n    Computes the logarithm of an integer `x` with an optional base.\n\n    Args:\n        x (int): The integer value for which the logarithm is computed. Must be greater than 0.\n\n    Returns:\n        float: The logarithm of `x` to the specified `base`.\n    ", "parameters": {"prop

In [10]:
REACT_SYSTEM_PROMPT = REACT_SYSTEM_PROMPT % tools_signature

In [11]:
print(REACT_SYSTEM_PROMPT)


You are a function calling AI model. You operate by running a loop with the following steps: Thought, Action, Observation.
You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don' make assumptions about what values to plug
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.

For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>, "id": <monotonically-increasing-id>}
</tool_call>

Here are the available tools / actions:

<tools> 
{"name": "sum_two_elements", "description": "\n    Computes the sum of two integers.\n\n    Args:\n        a (int): The first integer to be summed.\n        b (int): The second integer to be summed.\n\n    Returns:\n        int: The sum of `a` and `b`.\n    ", "parameters": {"pr

### ReAct Loop Step 1

In [12]:
USER_QUESTION = "I want to calculate the sum of 1234 and 5678 and multiply the result by 5. Then, I want to take the logarithm of this result"
chat_history = [
    {"role": "system", "content": REACT_SYSTEM_PROMPT},
    {"role": "user", "content": f"<question>{USER_QUESTION}</question>"},
]

In [13]:
output = (
    client.chat.completions.create(messages=chat_history, model=MODEL)
    .choices[0]
    .message.content
)

print(output)

<thought>I need to first calculate the sum of 1234 and 5678.</thought>
<tool_call>{"name": "sum_two_elements","arguments": {"a": 1234,"b": 5678}, "id": 0}</tool_call>


In [14]:
chat_history.append({"role": "assistant", "content": output})

### ReAct Loop Step 2

In [15]:
tool_call = extract_tag_content(output, tag="tool_call")

In [16]:
tool_call

TagContentResult(content=['{"name": "sum_two_elements","arguments": {"a": 1234,"b": 5678}, "id": 0}'], found=True)

In [17]:
tool_call = json.loads(tool_call.content[0])

In [18]:
tool_call

{'name': 'sum_two_elements', 'arguments': {'a': 1234, 'b': 5678}, 'id': 0}

In [19]:
tool_result = available_tools[tool_call["name"]].run(**tool_call["arguments"])

In [20]:
assert tool_result == 1234 + 5678

In [21]:
chat_history.append(
    {"role": "user", "content": f"<observation>{tool_result}</observation>"}
)

### ReAct Loop Step 3

In [22]:
output = (
    client.chat.completions.create(messages=chat_history, model=MODEL)
    .choices[0]
    .message.content
)

print(output)

<thought>Now that I have the sum of 1234 and 5678 which is 6912, I will multiply this result by 5.</thought>
<tool_call>{"name": "multiply_two_elements","arguments": {"a": 6912,"b": 5}, "id": 1}</tool_call>


In [23]:
chat_history.append({"role": "assistant", "content": output})

### ReAct Loop Step 4

In [24]:
tool_call = extract_tag_content(output, tag="tool_call")
tool_call = json.loads(tool_call.content[0])
tool_result = available_tools[tool_call["name"]].run(**tool_call["arguments"])

In [25]:
tool_result

34560

In [26]:
assert tool_result == (1234 + 5678) * 5

In [27]:
chat_history.append(
    {"role": "user", "content": f"<observation>{tool_result}</observation>"}
)

### ReAct Loop Step 5

In [28]:
output = (
    client.chat.completions.create(messages=chat_history, model=MODEL)
    .choices[0]
    .message.content
)

print(output)

<thought>Now that I have multiplied the result by 5 and obtained 34560, I will calculate the logarithm of this result.</thought>
<tool_call>{"name": "compute_log","arguments": {"x": 34560}, "id": 2}</tool_call>


In [29]:
chat_history.append({"role": "assistant", "content": output})

### ReAct Loop Step 6

In [30]:
tool_call = extract_tag_content(output, tag="tool_call")
tool_call = json.loads(tool_call.content[0])
tool_result = available_tools[tool_call["name"]].run(**tool_call["arguments"])

In [31]:
tool_result

10.450452222917992

In [32]:
assert tool_result == math.log((1234 + 5678) * 5)

In [33]:
chat_history.append(
    {"role": "user", "content": f"<observation>{tool_result}</observation>"}
)

### ReAct Loop Step 7

In [34]:
output = (
    client.chat.completions.create(messages=chat_history, model=MODEL)
    .choices[0]
    .message.content
)

print(output)

<response>The logarithm of the result is approximately 10.45.</response>


## Doing the same but with `agentic_patterns` library

In [38]:
from planning_pattern.react_agent import ReactAgent

In [39]:
agent = ReactAgent(
    client=client, tools=[sum_two_elements, multiply_two_elements, compute_log]
)

LLM: gpt-4o-mini and Model gpt-4o-mini


In [37]:
agent.run(
    user_msg="I want to calculate the sum of 1234 and 5678 and multiply the result by 5. Then, I want to take the logarithm of this result"
)


Thought: I need to perform the operations in the specified order: first calculate the sum, then multiply the result by 5, and finally compute the logarithm of the final result.

Using Tool: sum_two_elements

Tool call dict: 
{'name': 'sum_two_elements', 'arguments': {'a': 1234, 'b': 5678}, 'id': 0}

Tool result: 
6912

Observations: {0: 6912}

Thought: Now that I have the sum result of 6912, I will multiply this by 5.

Using Tool: multiply_two_elements

Tool call dict: 
{'name': 'multiply_two_elements', 'arguments': {'a': 6912, 'b': 5}, 'id': 1}

Tool result: 
34560

Observations: {1: 34560}

Thought: Now that I have the multiplication result of 34560, I will compute the logarithm of this result.

Using Tool: compute_log

Tool call dict: 
{'name': 'compute_log', 'arguments': {'x': 34560}, 'id': 2}

Tool result: 
10.450452222917992

Observations: {2: 10.450452222917992}


'The logarithm of the result, which is 34560, is approximately 10.45.'